In [4]:
import cv2
import numpy as np

def segment_mammary_region(image):
    # Aplicar limiarização adaptativa para segmentar a região da mama
    _, thresholded = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Encontrar os contornos dos objetos na imagem binarizada
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Criar uma máscara em branco do tamanho da imagem
    mask = np.zeros_like(image)

    # Encontrar o contorno da mama (maior área)
    largest_contour = max(contours, key=cv2.contourArea)

    # Desenhar o contorno da mama na máscara
    cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

    # Criar uma imagem em branco do tamanho da imagem original
    result_image = np.zeros_like(image)

    # Preencher a região da mama com branco (255)
    result_image[mask == 255] = 255

    return result_image

# Carregar a imagem em preto e branco
image = cv2.imread('S:/Desktop/PAI-github/mamografias/DleftCC/d_left_cc (1).png', cv2.IMREAD_GRAYSCALE)

# Segmentar a região da mama e remover a placa
segmented_image = segment_mammary_region(image)
resized_image = cv2.resize(segmented_image, (800, 600))  # Escolha as dimensões desejadas

# Mostrar a imagem resultante
cv2.imshow('Imagem resultante', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
